In [24]:
'''
Parameters:
    1. lowercase vs. uppercase
    2. categories are 1-vs-all, rather than 20 class softmax
    3. filter punctuation vs. keep punctuation as tokens
    4. GRU vs. LSTM
    4. adam vs. rmsprop
    6. Let the max_length limit to 500, rather than doing it in EDA
    
Tricky token parsing:
    count = [10731, 37877, 8771, 3451, 42795]
'''

'\nParameters:\n    1. lowercase vs. uppercase\n    2. categories are 1-vs-all, rather than 20 class softmax\n    3. filter punctuation vs. keep punctuation as tokens\n    4. GRU vs. LSTM\n    4. adam vs. rmsprop\n    6. Let the max_length limit to 500, rather than doing it in EDA\n    \nTricky token parsing:\n    count = [10731, 37877, 8771, 3451, 42795]\n'

In [5]:
from spacy.en import English
nlp = English()

In [6]:
import time as tm
import numpy as np
import pandas as pd
import re

verbose = False
np.random.seed(14)
input_csv = "data/snippets.csv"
full = pd.read_csv(input_csv) # , header=None)
print full.shape

full = full.dropna(axis=0)
print full.shape

full = full[full['Content'].str.len() < 600]
print full.shape

(65499, 3)
(64458, 3)
(64434, 3)


In [7]:
X = full['Content']
part1 = X.apply(lambda row: row.strip())
part2 = part1.apply(lambda row: nlp(unicode(row, 'utf-8', 'ignore')) )
part3 = part2.apply(lambda row: [token.orth_ for token in row])
X = part3

In [8]:
print X.head()
print X[4]
print len(X[4])
print type(X[4])

0    [Boomerang, Commerce, ,, creators, of, the, fi...
1    [Starship, Technologies, ,, the, new, venture,...
2    [The, firm, will, launch, a, beta, program, in...
3    [PingPad, has, added, task, management, to, it...
4    [But, with, this, latest, release, ,, the, goa...
Name: Content, dtype: object
[u'But', u'with', u'this', u'latest', u'release', u',', u'the', u'goal', u'is', u'to', u'better', u'organize', u'roles', u'and', u'responsibilities', u'for', u'projects', u',', u'similar', u'to', u'what', u'you', u"'d", u'receive', u'in', u'Trello', u'and', u'other', u'task', u'management', u'offerings', u'.']
32
<type 'list'>


In [18]:
print X[8771]
print X[10731]
print X[3451]

[u'[', u'Update', u'3:30pm', u'PST', u':', u'Another', u'source', u'now', u'tells', u'us', u'LiveRail', u'sold', u'for', u'$', u'500', u'million', u',', u'matching', u'the', u'$', u'400', u'million', u'to', u'$', u'500', u'million', u'range', u'I', u'reported', u'earlier', u'.', u']']
[u'\u201c', u'If', u'you', u'are', u'selling', u'a', u'product', u'for', u'$', u'12', u'and', u'you', u'see', u'your', u'competitor', u'is', u'selling', u'it', u'for', u'$', u'11.50', u'so', u'you', u'drop', u'your', u'price', u'to', u'$', u'11.49', u'you', u'have', u'not', u'accomplished', u'anything', u'except', u'starting', u'a', u'price', u'war', u',', u'\u201d', u'he', u'says', u'.']
[u'FirstBank', u"'s", u'net', u'income', u'for', u'full', u'-', u'year', u'2015', u'increased', u'to', u'$', u'177.9', u'million', u'from', u'$', u'175.4', u'million', u'in', u'2014', u'.']


In [20]:
y = full['Tags']

y = y.str.split(';').str[0]
print y.value_counts()

strategy                  14376
funding                    8385
overview                   6224
revenue                    3932
partnership                3661
acquisition                3432
headcount                  3119
users                      3082
key_person_hire            3014
turmoil                    2313
product                    1654
valuation                  1347
award                      1188
market                     1168
founding                    913
customers                   820
sector                      808
key_person_departure        788
total-funding               777
ipo                         762
competitors                 720
lawsuit                     527
office_move                 403
geo                         378
reorg                       190
patent                       90
conference-sponsorship       82
executive-promotion          69
bankruptcy                   69
filing-ipo                   65
event-participant            52
scale   

In [21]:
y = y.str.replace('total-', '')
y = y.str.replace('filing-', '')
y = y.str.replace('key_person_', '')

y = y.str.replace('lawsuit', 'OTHER')
y = y.str.replace('office_move', 'OTHER')
y = y.str.replace('geo', 'OTHER')
y = y.str.replace('reorg', 'OTHER')
y = y.str.replace('patent', 'OTHER')
y = y.str.replace('conference-sponsorship', 'OTHER')
y = y.str.replace('executive-promotion', 'OTHER')
y = y.str.replace('bankruptcy', 'OTHER')
y = y.str.replace('event-participant', 'OTHER')
y = y.str.replace('scale', 'OTHER')
y = y.str.replace('team-grew', 'OTHER')
y = y.str.replace('executive-publication', 'OTHER')
y = y.str.replace('job-posting', 'OTHER')
y = y.str.replace('leadership-change', 'OTHER')

y = y.fillna('OTHER')
jake = y.value_counts()
print len(jake)
print jake

21
strategy       14376
funding         9162
overview        6224
revenue         3932
partnership     3661
acquisition     3432
headcount       3119
users           3082
hire            3014
turmoil         2313
OTHER           1886
product         1654
valuation       1347
award           1188
market          1168
founding         913
ipo              827
customers        820
sector           808
departure        788
competitors      720
dtype: int64


In [22]:
print y.shape
y.name="Tags"
print X.shape
ser = [X, y]
data = pd.concat([X, y], axis=1)
print data.shape

(64434,)
(64434,)
(64434, 2)


In [25]:
print data['Content'][8771]
print data['Content'][10731]
print data['Content'][3451]

[u'[', u'Update', u'3:30pm', u'PST', u':', u'Another', u'source', u'now', u'tells', u'us', u'LiveRail', u'sold', u'for', u'$', u'500', u'million', u',', u'matching', u'the', u'$', u'400', u'million', u'to', u'$', u'500', u'million', u'range', u'I', u'reported', u'earlier', u'.', u']']
[u'\u201c', u'If', u'you', u'are', u'selling', u'a', u'product', u'for', u'$', u'12', u'and', u'you', u'see', u'your', u'competitor', u'is', u'selling', u'it', u'for', u'$', u'11.50', u'so', u'you', u'drop', u'your', u'price', u'to', u'$', u'11.49', u'you', u'have', u'not', u'accomplished', u'anything', u'except', u'starting', u'a', u'price', u'war', u',', u'\u201d', u'he', u'says', u'.']
[u'FirstBank', u"'s", u'net', u'income', u'for', u'full', u'-', u'year', u'2015', u'increased', u'to', u'$', u'177.9', u'million', u'from', u'$', u'175.4', u'million', u'in', u'2014', u'.']


In [32]:
data.to_pickle('data/training_data.p')